# Task 2


In [3]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

Load the dataset

In [ ]:
images = np.load('../a2_data/images.npy')
labels = np.load('../a2_data/labels.npy')


#print the first few images with their labels
fig, axes = plt.subplots(3,3, figsize=(10,10))
for i in range(9):
    ax = axes[i//3, i%3]
    ax.imshow(images[i], cmap='gray')
    ax.set_title(labels[i])
    plt.tight_layout()

We can see that the dataset consists of images of clocks, each with a different angle, rotation and lightning. This makes it rather difficult to train a model to classify the images.

We will be trying multiple ways of labelling the clocks, firstly try by regression label.

In [ ]:
regressionLabels = np.zeros(labels.shape[0])
for i, (h, m) in enumerate(labels):
    regressionLabels[i] = h + m/60

X_train, X_test, y_train, y_test = train_test_split(images, regressionLabels, test_size=0.2, random_state=42)

In [40]:
def lossFunction(y_true, y_pred):
    #convert tensor to numpy array
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    return np.min(tf.math.abs(y_true - y_pred), 12 - tf.math.abs(y_true - y_pred))

In [4]:
def activationFunction(x):
    return keras.activations.relu(x, alpha=0.0, max_value=12)

In [6]:
a = tf.constant([-3, -1, 0, 1 , 3, 100])
b = activationFunction(a)
tf.print(b)

[0 0 0 1 3 12]


In [42]:
#build a cnn model
model = keras.Sequential([

    layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,1)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(1, activation='sigmoid'),
    layers.Dropout(0.5),
    layers.Dense(1, activation=activationFunction)
    ])s

AttributeError: module 'keras.api._v2.keras.layers' has no attribute 'convolutional'

In [39]:
model.compile(loss=lossFunction, optimizer='adam', metrics=['mean_squared_error'])

history = model.fit(X_train, y_train, batch_size=32, epochs=3, validation_split=0.1, verbose=1)

#model.evaluate(X_test, y_test, verbose=2)

#show differences between predicted and actual values
# add straight line to show perfect prediction

y_pred = model.predict(X_test)
plt.figure(figsize=(10,10))
plt.scatter(y_test, y_pred)
plt.plot([0, 24], [0, 24], color='red')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.show()

Epoch 1/3
 11/405 [..............................] - ETA: 3:11 - loss: 52.5282 - mean_squared_error: 52.5282

KeyboardInterrupt: 